# Wind Power Prediciotn using Time Series


<h1><center><a href=https://colab.research.google.com/github/Nemczek/wind_power_prediction/blob/main/wind_power_prediction_time_series.ipynb>Open in Colab!</a></center></h1>

<center><img src='windmills.jpg'></center>

Continued prediction of Wind Power Generation data set using time series

Note: You can run notebook directly in your browser by replacing .com with .dev in github link.
Example: [https://github.dev/Nemczek/wind_power_prediction](https://github.dev/Nemczek/wind_power_prediction_time_series)

## Packages 

In [ ]:
# Run only once - installing packages. Needed to run notebook in browser
install.packages(c("forecast", "tempdisagg", "tidyverse"))

In [ ]:
library(forecast)
library(tempdisagg)
library(tidyverse)

## Data loading and cleaning/prepering 

In [ ]:
dane <- read.csv("Location1.csv")

### Aggregate data into monthly data and load and split into test and teaching data

In [ ]:
dane$Time <- as.Date(dane$Time)
dane$Miesiac <- format(dane$Time, "%Y-%m")

dane.fan <- dane %>%
  group_by(Miesiac) %>%
  summarise(srednia_Power = mean(Power))

dane.fan <- data.frame(dane.fan$srednia_Power)

### Cleaning data and looking for Trend and seasonality

In [ ]:

fan <- ts(dane.fan, start = c(2017,1), frequency = 12)
ts.plot(fan)
monthplot(fan)
seasonplot(fan, year.labels = T, col = 1:5)
lag.plot(fan, do.lines = F, lags = 12) # There is T i S
Acf(fan)
Pacf(fan)
tsoutliers(fan)
fan[13] <- 0.4186132 # 20018-01 


fan.ts <- window(fan, end = c(2020,12))
ts.plot(fan.ts)
monthplot(fan.ts)
seasonplot(fan.ts, year.labels = T, col = rainbow(4))
lag.plot(fan.ts, do.lines = F, lags = 12) # there is T i S
Acf(fan.ts)
Pacf(fan.ts)
tsoutliers(fan.ts)
fan[28] <- 0.3541302 # 2019-02


fan.test <- window(fan, start = c(2021,1))
tsoutliers(fan.test)

rm(dane,dane.fan,fan)

## Manual differentiation 

In [ ]:
fan.ts.diff <- diff(fan.ts, lag = 12)
ts.plot(fan.ts.diff)
lag.plot(fan.ts.diff, do.lines = F, lags = 12)
Acf(fan.ts.diff) #MA(3)
Pacf(fan.ts.diff) #AR(3)

### MA(3)

In [ ]:
model.MA3 <- Arima(y =fan.ts, order = c(0,0,3), seasonal = c(0,1,0))
summary(model.MA3)

coefs = model.MA3$coef
coefs.sd = sqrt(diag(model.MA3$var.coef))
ind = abs(coefs/(1.96*coefs.sd)) # ratio
signif = which(ind >= 1 )
temp.fixed = numeric(3)
temp.fixed[signif] = NA
model.MA3.signif = Arima( y = fan.ts, order = c(0,0,3), seasonal = c(0,1,0), fixed = temp.fixed )
summary(model.MA3.signif) #No significant coefficients 

SARIMA(0,0,0)(0,1,0)[12] model can be done due to the non-significance of the MA(3) coefficient

In [ ]:

model.0 <- Arima(fan.ts, order = c(0,0,0), seasonal = c(0,1,0))
summary(model.0)
prog.model.0 <- forecast(model.0, h = 12)
prog.model.0$mean
ts.plot(prog.model.0$mean)
plot(prog.model.0)
lines(fan.test, col = "red")
Acc0 <- accuracy(prog.model.0, fan.test)[,c("ME","MAE","MPE",
                                "MAPE", "RMSE", "Theil's U")]

### AR(3)

In [ ]:
model.AR3 <- Arima(fan.ts, order = c(3,0,0), seasonal = c(0,1,0))
summary(model.AR3)


coefs = model.AR3$coef
coefs.sd = sqrt(diag(model.AR3$var.coef))
ind = abs(coefs/(1.96*coefs.sd)) # ratio
signif = which(ind >= 1 )
temp.fixed = numeric(3)
temp.fixed[signif] = NA
model.AR3.signif = Arima( y = fan.ts, order = c(3,0,0), seasonal = c(0,1,0), fixed = temp.fixed )
summary(model.AR3.signif) ##significant coefficients: AR(3) 

Forecast

In [ ]:
prog.AR3 <- forecast(model.AR3.signif, h = 12)
prog.AR3$mean
ts.plot(prog.AR3$mean)
plot(prog.AR3)
lines(fan.test, col = "red")
AccAR3 <- accuracy(prog.AR3, fan.test)[,c("ME","MAE","MPE",
                                          "MAPE", "RMSE", "Theil's U")]

## AutoArima

For comparison the auto.arima was used

In [ ]:
fan.ts.auto <- auto.arima(fan.ts)
summary(fan.ts.auto) #ARIMA(0,0,0)(0,1,0)[12]#


We got ARIMA(0,0,0)(0,1,0)[12] model

Forecast for auto.arima model 

In [ ]:
fan.prog.auto <- forecast(fan.ts.auto, h = 12)
fan.prog.auto$mean
ts.plot(fan.prog.auto$mean)
plot(fan.prog.auto)
lines(fan.test, col = "red")
accuracy(fan.prog.auto, fan.test)[,c("ME","MAE","MPE",
                                     "MAPE", "RMSE", "Theil's U")]

## Classical decomposition 

In [ ]:
fan.decom <- decompose(fan.ts)
plot(fan.decom)
fan.decom.res <- fan.decom$random
Acf(fan.decom.res) #MA(12)
Pacf(fan.decom.res) #AR(2)


### Models for classical decomposition

In [ ]:
model.decom.MA12 <- Arima(fan.ts, order = c(0,0,12), seasonal = c(0,1,0))
summary(model.decom.MA12)

In [ ]:
model.decom.AR2 <- Arima(fan.ts, order = c(2,0,0), seasonal = c(0,1,0))
summary(model.decom.AR2)

#### MA(12)

In [ ]:
coefs = model.decom.MA12$coef
coefs.sd = sqrt(diag(model.decom.MA12$var.coef))
ind = abs(coefs/(1.96*coefs.sd)) # ratio
signif = which(ind >= 1 )
temp.fixed = numeric(12)
temp.fixed[signif] = NA
model.decom.1.signif = Arima( y = fan.ts, order = c(0,0,12), seasonal = c(0,1,0), fixed = temp.fixed )
summary(model.decom.1.signif)# No significant coefficients

model Identical to  SARIMA(0,0,0)(0,1,0)

#### AR(2)

In [ ]:

#AR(2)
coefs = model.decom.AR2$coef
coefs.sd = sqrt(diag(model.decom.AR2$var.coef))
ind = abs(coefs/(1.96*coefs.sd)) # ratio
signif = which(ind >= 1 )
temp.fixed = numeric(2)
temp.fixed[signif] = NA
model.decom.2.signif = Arima( y = fan.ts, order = c(2,0,0), seasonal = c(0,1,0), fixed = temp.fixed )
summary(model.decom.2.signif)# No significant coefficients

model Identical to  SARIMA(0,0,0)(0,1,0)